In [1]:
import pandas as pd
import sys

sys.path.append("..")

from src.auth.aurora import get_portal_client

In [2]:
portal_client = get_portal_client()

In [3]:
existing_missing_df = pd.read_csv("data/2025-08-04 Masterfire Missing Tasks New 2024.csv").sort_values("site_id")

In [4]:
missing_sites = existing_missing_df["site_id"].unique().tolist()

In [5]:
placeholders = ', '.join(['%s'] * len(missing_sites))
site_df = portal_client.query_to_pandas(
    f"""SELECT site.id AS site_id, site.name AS equipment_name, client.name AS site_name
    FROM topaz.api_sites AS site
    LEFT JOIN topaz.api_clients AS client
    ON site.client_id = client.id
    WHERE site.id IN ({placeholders})
    ORDER BY site.id""",
    params=missing_sites
)

In [ ]:
result_df = pd.read_csv("data/Masterfire Problem Services.csv")

In [ ]:
for _, (site_id, equipment_name, site_name) in site_df[["site_id", "equipment_name", "site_name"]].iterrows():
    print(site_id)
    problem_tests = portal_client.query_to_list(
        """SELECT
            DISTINCT mode_change.created_at
        FROM topaz.api_service_tests AS test
        INNER JOIN topaz.api_mode_changes AS mode_change
        ON test.mode_change_id = mode_change.id
        INNER JOIN topaz.api_sites AS site
        ON test.site_id = site.id
        INNER JOIN topaz.api_toc_rows AS toc
        ON (mode_change.toc_id = toc.toc_id)
        AND (
            (test.domain = toc.domain) OR
            (toc.domain IS NULL)
        )
        AND (
            (test.panel = toc.panel) OR
            (toc.panel IS NULL)
        )
        AND (
            (test.loop = toc.loop) OR
            (toc.loop IS NULL)
        )
        AND (test.address = toc.address)
        AND (test.address_type = toc.address_type)
        AND (
            (
                (
                    (test.sub_address = '') OR
                    (test.sub_address IS NULL)
                ) AND (
                    (toc.sub_address IS NULL) OR
                    (toc.sub_address = 0)
                )
            ) OR
            (test.sub_address = toc.sub_address)
        )
        AND test.created_at BETWEEN toc.created_at AND IFNULL(toc.deleted_at, CURRENT_TIMESTAMP())
        INNER JOIN topaz.api_service_tasks AS task_template
        ON task_template.template_id = site.template_id
        AND (
            (
                (task_template.asset_type = toc.lcs_device_type)
                AND (toc.lcs_device_type!= 'Other')
            )
            OR (
                toc.lcs_device_type = 'Other'
                AND (toc.type = task_template.asset_type)
            )
            OR (task_template.asset_type = '')
            OR (
                site.protocol_id = 34
                AND asset_type = 'el'
            )
            OR (
                site.protocol_id = 35
                AND asset_type = 'ext'
                AND toc.lcs_device_type != 'Other'
                AND toc.type != 'Fire Blanket'
            )
        )
        AND test.created_at BETWEEN task_template.created_at AND IFNULL(task_template.deleted_at, CURRENT_TIMESTAMP())
        AND task_template.type IN (1,3)
        LEFT JOIN topaz.api_service_tasks_record AS task_record
        ON task_record.task_id = task_template.id
        AND task_record.test_id = test.id
        WHERE task_record.id IS NULL
        AND test.created_at >= '2024-01-01 00:00:00'
        AND site.id = %(site_id)s
        """,
        {"site_id": site_id}
    )
    assert len(problem_tests) > 0, f"No problem tests found for site {site_id} ({equipment_name})"
    for created_at in problem_tests:
        result_df = pd.concat([
            result_df,
            pd.DataFrame({
                "Equipment Name": [equipment_name],
                "Site Name": [site_name],
                "Service Start Time": [created_at]
            })
        ], ignore_index=True)
        result_df.to_csv("data/Masterfire Problem Services.csv", index=False)


8466


[datetime.datetime(2024, 9, 24, 7, 24, 47)]